# Kubernetes

# Kubernetes Version (Current)

In [43]:
from kubernetesTool import executeKubectlCmd
from kubernetesTool import toCmd
from kubernetesTool import executeLocalCmd
from kubernetesTool import createConfig
from kubernetesTool import getPort

## addPeer

In [44]:
def addPeer(peername):  
    if peername in peers.keys():
        return peers[peername]
    # Create configuration for new peer
    executeLocalCmd(['/bin/bash','-c','cd /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/ && /home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/configure-deployment.sh -n ' + peername])
    # Start service
    executeKubectlCmd(['apply', '-f', '/home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/target/' + peername + '/k8s-peer-service.yaml'])
    # Create peer configuration
    res = createConfig(peername=peername,
                       peer_listen_port=getPort(peername=peername, name='peer-listen'),
                       peer_gossip_port=getPort(peername=peername, name='peer-gossip'))
    # Start new peer
    executeKubectlCmd(['apply', '-f','/home/jovyan/scray/projects/invoice-hyperledger-fabric/containers/target/' + peername + '/k8s-peer.yaml'])
    peer = kubernetespeer(name=peername)
    peers[peername] = peer
    return peer
    
# create, delete numbered peers
def createPeers(base_name,start=0, end=10):
    for x in range(start,end):
        peername = base_name + '-' + str(x)
        addPeer(peername)
         
def deletePeers(name):
    for x in range(0,10):
        peerneme = name + str(x)
        deletePeer(peerneme)    
  
def deleteAllPeers(config):
    for item in config['items']:
        try:
            if 'org_name' in item['data'].keys():
                deletePeer(item['data']['org_name'])
        except Exception as e:
            print(item['data'])
            
#createPeers('oly')
#deletePeers('ga')

In [45]:
import subprocess
def deleteConfig(peername=''):
    try:  
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'delete','configmap','hl-fabric-peer-' + peername])
        return str(output)
    except Exception as e:
        return str(e)  
        
    
def createConfig(peername='',peer_listen_port='',peer_gossip_port=''):
    try:  
        peer_host_name = peername + '.kubernetes.research.dev.seeburger.de'
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'create','configmap','hl-fabric-peer-' + peername,
                                          '--from-literal=hostname=' + peer_host_name,
                                          '--from-literal=org_name=' + peername,
                                          '--from-literal=data_share=hl-fabric-data-share-service:80',
                                          '--from-literal=CORE_PEER_ADDRESS=peer0.' + peer_host_name + ':' + peer_listen_port,
                                          '--from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=' + peer_host_name + ':' + peer_gossip_port,
                                          '--from-literal=CORE_PEER_LOCALMSPID=' + peername + 'MSP'])
        return str(output)
    except Exception as e:
        return str(e)  

In [46]:
def executePeerCmd(pod_name,cmd, cli='scray-peer-cli'):
    try:  
        #cmd1 = "source /root/.bashrc && "
        cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', pod_name, '-c', cli,'--', '/bin/sh','-c',cmd]
        #cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', pod_name, '-c', cli,'--', '/bin/sh','-c'] + cmd
        print(toCmd(cmd1))
        output  = subprocess.check_output(cmd1)
        #output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'exec', '--stdin' ,'--tty', pod_name, '-c', cli,'--', '/bin/sh','-c',cmd])
        return output.decode('ascii')
    except Exception as e:
        return str(e)

In [47]:
import kubernetesTool

class kubernetespeer:
    def __init__(self, name='',prefix='peer'):
        self.name = name
        self.pod_name = kubernetesTool.getPod(name)
        self.listen_port = kubernetesTool.getPort(peername=name, name = prefix + '-listen')
        self.gossip_port = kubernetesTool.getPort(peername=name, name = prefix + '-gossip')
        # ????????
        #print('kubernetespeer addBashEnvToPeer')
        #addBashEnvToPeer(self)
    def getCertificate(self):
        try:
            return str(subprocess.check_output(['cat', self.crt]))[2:-3]   
        except Exception as e:
            return str(e)      
        
class ordererpeer(kubernetespeer):
    def __init__(self, name='',prefix='peer'):
        super().__init__(name,prefix)
        self.host = 'orderer.example.com'
        self.orderer_intern = 7050
        
    # can change
    def get_ip(self, app='orderer-org1-scray-org'):
        return executeKubectlCmd(['get', 'pods','-l', 'app=' + app,'-o','jsonpath=\'{.items[*].status.podIP}\''])   
    

In [48]:
def toCmd(strlist):
    return ''.join((str(e) + ' ') for e in strlist)[:-1]


#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $CHANNEL_NAME         
def joinHyperledger(peer,orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host,ext_peer_ip='10.15.136.41'):          
    cmd = toCmd(['/mnt/conf/peer_join.sh', orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host,ext_peer_ip])
    print(cmd)
    return executePeerCmd(peer.pod_name, cmd)

def endorse_peer(peer=None, orderer_ip='',channel_name='',shared_fs_host='',ext_peer_ip='10.15.136.41'):
    print(peer.name)
    peername = peer.name
    peer_host_name = peername + '.kubernetes.research.dev.seeburger.de'
    cmd = toCmd(['/mnt/conf/orderer/scripts/inform_existing_nodes.sh',orderer_ip,channel_name,peer.name,shared_fs_host,ext_peer_ip,peer_host_name]) 
    #print(cmd)
    return executePeerCmd(orderer.pod_name,cmd,cli='scray-orderer-cli')

def add_channel(name):
    cmd1 = ['/mnt/conf/orderer/scripts/create_channel.sh', name, 'orderer.example.com', '7050']
    cmd = toCmd(cmd1)
    return executePeerCmd(orderer.pod_name, cmd1, cli='scray-orderer-cli')

## check status peers

In [107]:
from kubernetesTool import executeKubectlCmd
_res = executeKubectlCmd(['get', 'pods', '-o', 'json'], decode='json')

for item in _res['items']:
    labels=item['metadata']['labels']
    #status=item['status']['initContainerStatuses'][0]
    #print(labels,status['state'],status['restartCount'],status['name'],status['ready'])
    print(labels,item['status']['phase'])
    #print(labels)

{'app': 'hl-fabric-data-share', 'pod-template-hash': '6849487f76'} Running
{'app': 'invoice-chaincode-external', 'pod-template-hash': '847cd44954'} Running
{'app': 'lalya-0', 'pod-template-hash': '7c887cffb8'} Running
{'app': 'lalya-1', 'pod-template-hash': '7c76dff8c'} Running
{'app': 'lalya-2', 'pod-template-hash': '5f8b45dbc4'} Running
{'app': 'orderer-org1-scray-org', 'pod-template-hash': '9dbd9fc68'} Running
{'app': 'peer48', 'pod-template-hash': '57445d5f7b'} Running


In [74]:
_res['items'][3]['metadata']['labels']
_res['items'][0]['status']['phase']

'Running'

## create peers

In [49]:
peers = {}

In [78]:
peers

{'lalya-0': <__main__.kubernetespeer at 0x7f7dac574640>,
 'lalya-1': <__main__.kubernetespeer at 0x7f7dac536700>,
 'lalya-2': <__main__.kubernetespeer at 0x7f7dac536730>}

In [81]:
orderer = ordererpeer(name='orderer-org1-scray-org',prefix='orderer')

In [51]:
createPeers('lalya',start=0, end=3)

In [ ]:
orderer.__dict__

In [ ]:
kubectl exec --stdin --tty $ORDERER_POD -c scray-orderer-cli  -- /bin/sh /mnt/conf/orderer/scripts/inform_existing_nodes.sh $ORDERER_IP $CHANNEL_NAME $PEER_NAME $SHARED_FS_HOST $EXT_PEER_IP $PEER_HOST_NAME

In [ ]:
EER_POD_NAME=$(kubectl get pod -l app=$PEER_NAME -o jsonpath="{.items[0].metadata.name}")
 ORDERER_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
 ORDERER_PORT=7050
 PEER_PORT=$(kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-listen')].nodePort}")
 kubectl exec --stdin --tty $PEER_POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $ORDERER_PORT $CHANNEL_NAME $SHARED_FS_HOST $EXT_PEER_IP

In [99]:
#orderer_ip=get_orderer_ip('orderer-org1-scray-org') 
#orderer_hostname='orderer.example.com'
#orderer_port='7050'

orderer_ip=orderer.get_ip()
orderer_hostname=orderer.host
#orderer_port=orderer.orderer_intern
orderer_port= orderer.listen_port

channel_name='super43'
shared_fs_host='10.14.128.38:30080'
    
def endorse_and_join(peer):    
    res1 = endorse_peer(peer=peer,orderer_ip=orderer_ip,channel_name=channel_name,shared_fs_host='10.14.128.38:30080')   
    res2  = joinHyperledger(peer,orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host)
    return res1,res2

for key, value in peers.items():
    endorse_and_join(value)

lalya-0
/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-9dbd9fc68-5lrgq -c scray-orderer-cli -- /bin/sh -c /mnt/conf/orderer/scripts/inform_existing_nodes.sh '10.32.0.5' super43 lalya-0 10.14.128.38:30080 10.15.136.41 lalya-0.kubernetes.research.dev.seeburger.de


2021-10-07 07:55:57.873 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:55:57.899 UTC [cli.common] readBlock -> INFO 002 Received block: 3
2021-10-07 07:55:57.900 UTC [channelCmd] fetch -> INFO 003 Retrieving last config block: 3
2021-10-07 07:55:57.919 UTC [cli.common] readBlock -> INFO 004 Received block: 3
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   237  100   237    0     0  10304      0 --:--:-- --:--:-- --:--:-- 10304
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   847    0     0  100   847      0  44578 --:--:-- --:--:-- --:--:-- 44578
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
10

/mnt/conf/peer_join.sh '10.32.0.5' orderer.example.com 30081 super43 10.14.128.38:30080 10.15.136.41
/home/jovyan/work/usr/bin/kubectl exec -t lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer_join.sh '10.32.0.5' orderer.example.com 30081 super43 10.14.128.38:30080 10.15.136.41


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   847  100   847    0     0   103k      0 --:--:-- --:--:-- --:--:--  103k
2021-10-07 07:55:59.552 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:55:59.563 UTC [cli.common] readBlock -> INFO 002 Received block: 0
2021-10-07 07:55:59.631 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:55:59.819 UTC [channelCmd] executeJoin -> INFO 002 Successfully submitted proposal to join channel


lalya-1
/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-9dbd9fc68-5lrgq -c scray-orderer-cli -- /bin/sh -c /mnt/conf/orderer/scripts/inform_existing_nodes.sh '10.32.0.5' super43 lalya-1 10.14.128.38:30080 10.15.136.41 lalya-1.kubernetes.research.dev.seeburger.de


...

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   847  100   847    0     0   103k      0 --:--:-- --:--:-- --:--:--  103k
2021-10-07 07:56:01.762 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:56:01.771 UTC [cli.common] readBlock -> INFO 002 Received block: 0
2021-10-07 07:56:01.852 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:56:01.971 UTC [channelCmd] executeJoin -> INFO 002 Successfully submitted proposal to join channel


lalya-2
/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-9dbd9fc68-5lrgq -c scray-orderer-cli -- /bin/sh -c /mnt/conf/orderer/scripts/inform_existing_nodes.sh '10.32.0.5' super43 lalya-2 10.14.128.38:30080 10.15.136.41 lalya-2.kubernetes.research.dev.seeburger.de


2021-10-07 07:56:02.337 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:56:02.349 UTC [cli.common] readBlock -> INFO 002 Received block: 3
2021-10-07 07:56:02.349 UTC [channelCmd] fetch -> INFO 003 Retrieving last config block: 3
2021-10-07 07:56:02.356 UTC [cli.common] readBlock -> INFO 004 Received block: 3
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   237  100   237    0     0  59250      0 --:--:-- --:--:-- --:--:-- 59250
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   847    0     0  100   847      0   165k --:--:-- --:--:-- --:--:--  165k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
10

/mnt/conf/peer_join.sh '10.32.0.5' orderer.example.com 30081 super43 10.14.128.38:30080 10.15.136.41
/home/jovyan/work/usr/bin/kubectl exec -t lalya-2-5f8b45dbc4-52tt7 -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer_join.sh '10.32.0.5' orderer.example.com 30081 super43 10.14.128.38:30080 10.15.136.41


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   847  100   847    0     0   137k      0 --:--:-- --:--:-- --:--:--  137k
2021-10-07 07:56:03.977 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:56:03.992 UTC [cli.common] readBlock -> INFO 002 Received block: 0
2021-10-07 07:56:04.051 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:56:04.165 UTC [channelCmd] executeJoin -> INFO 002 Successfully submitted proposal to join channel


In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-5f97c57d44-j5mbs -c scray-orderer-cli -- /bin/sh -c /mnt/conf/orderer/scripts/inform_existing_nodes.sh '10.32.0.5' super40 kalya-0 10.14.128.38:30080 10.15.136.41 kalya-0.kubernetes.research.dev.seeburger.de

In [ ]:
type(peers)

## install chaincode

In [94]:
channel_name
orderer.__dict__

{'name': 'orderer-org1-scray-org',
 'pod_name': 'orderer-org1-scray-org-9dbd9fc68-5lrgq',
 'listen_port': '30081',
 'gossip_port': '',
 'host': 'orderer.example.com',
 'orderer_intern': 7050}

In [100]:
#cc_host chaincode host
def installAndAproveChaincode(peer,orderer,channel_name,cc_host='10.14.128.38'):
    #!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK
    #strlist = ['/mnt/conf/install_and_approve_cc.sh', cc_host, orderer.get_ip(), orderer.host, str(orderer.orderer_intern), channel_name ]
    strlist = ['/mnt/conf/install_and_approve_cc.sh', cc_host, orderer.get_ip(), orderer.host, str(orderer.listen_port), channel_name ]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 
     
for key, peer in peers.items():
    installAndAproveChaincode(peer,orderer,channel_name)

/home/jovyan/work/usr/bin/kubectl exec -t lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh -c /mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super43


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   450  100   450    0     0   6081      0 --:--:-- --:--:-- --:--:--  6081
Error: chaincode install failed with status: 500 - failed to invoke backing implementation of 'InstallChaincode': chaincode already successfully installed
2021-10-07 07:56:32.992 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [663ae957435c3928f4673b317fd44928a73dc53ca00e49f49c235a7357439396] committed with status (VALID) at peer0.lalya-0.kubernetes.research.dev.seeburger.de:30700


/home/jovyan/work/usr/bin/kubectl exec -t lalya-1-7c76dff8c-pzx78 -c scray-peer-cli -- /bin/sh -c /mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super43


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   450  100   450    0     0   6521      0 --:--:-- --:--:-- --:--:--  6521
Error: chaincode install failed with status: 500 - failed to invoke backing implementation of 'InstallChaincode': chaincode already successfully installed
2021-10-07 07:56:35.983 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [42d14bc33c7343a67aff03cf2c57cee6ba6713585ecf92ce7d7f23cd30e55e13] committed with status (VALID) at peer0.lalya-1.kubernetes.research.dev.seeburger.de:32294


/home/jovyan/work/usr/bin/kubectl exec -t lalya-2-5f8b45dbc4-52tt7 -c scray-peer-cli -- /bin/sh -c /mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super43


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   450  100   450    0     0   7142      0 --:--:-- --:--:-- --:--:--  7142
Error: chaincode install failed with status: 500 - failed to invoke backing implementation of 'InstallChaincode': chaincode already successfully installed
2021-10-07 07:56:38.818 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [37653ed57eeeae1fa4b83ce55ce4f8bc8cd2b5e6ccfb471d239df6307752b6d7] committed with status (VALID) at peer0.lalya-2.kubernetes.research.dev.seeburger.de:30851


In [101]:
def commitChaincode(peer,channel_name):
    strlist = ['/mnt/conf/peer/cc_commit.sh', channel_name ]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

peer = list(peers.values())[0]
commitChaincode(peer,channel_name)

/home/jovyan/work/usr/bin/kubectl exec -t lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/cc_commit.sh super43


2021-10-07 07:57:09.677 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [782c14433b3559febd81e46c921fe1fbcb24a5dda8f240bbbceb223bf52a017f] committed with status (VALID) at peer0.lalya-0.kubernetes.research.dev.seeburger.de:30700


"Installed chaincode\nInstalled chaincodes on peer:\nPackage ID: basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee, Label: basic_1.0\nChaincode definition for chaincode 'basic', version '1.0', sequence '1' on channel 'super43' approval status by org:\nAdminOrgMSP: false\nlalya-0MSP: true\nlalya-1MSP: true\nlalya-2MSP: true\n"

In [105]:
peer.name, channel_name

('lalya-0', 'super43')

In [106]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh /mnt/conf/peer/cc-basic-interaction.sh  super43

Error: endorsement failure during invoke. response: status:500 message:"error in simulation: failed to execute transaction 42d0040afd7a8c8a129f481f503bb8828b87eceed4a967b663d8c0f4e3a017fe: could not launch chaincode basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee: connection to basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee failed: error cannot create connection for basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee: error creating grpc connection to asset-transfer-basic.org1.example.com:30999: failed to create new connection: connection error: desc = \"transport: error while dialing: dial tcp 10.14.128.38:30999: connect: connection refused\"" 
Error: endorsement failure during query. response: status:500 message:"error in simulation: failed to execute transaction 6baa63b319b0947a9c9d3f3c748efc7ac1681b5c6330dd441a0258232e2a59e6: could not launch chaincode basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd

In [108]:
#Call init method in chaincode
def chaincode_init(peer,channel_name):
    strlist = ['/mnt/conf/peer/cc-basic-interaction.sh', channel_name ]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

chaincode_init(peer,channel_name)

/home/jovyan/work/usr/bin/kubectl exec -t lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/cc-basic-interaction.sh super43


2021-10-07 09:08:07.889 UTC [chaincodeCmd] chaincodeInvokeOrQuery -> INFO 001 Chaincode invoke successful. result: status:200 


'[{"Key":"invoice1","Record":{"ID":"invoice1","InvoiceOwner":"x509::CN=User1@lalya-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-0.kubernetes.research.dev.seeburger.de,O=lalya-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE","ProductByer":"x509::CN=User1@lalya-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-0.kubernetes.research.dev.seeburger.de,O=lalya-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE"}}]\n'

In [ ]:
CHANNEL_NAME=mychannel
#ORDERER_NAME=orderer.example.com
IP_CC_SERVICE=10.14.128.38         # Host where the chaincode is running
#PEER_POD=$(kubectl get pod -l app=$PEER_NAME -o jsonpath="{.items[0].metadata.name}")
#ORDERER_IP=$(kubectl get pods  -l app=orderer-org1-scray-org -o jsonpath='{.items[*].status.podIP}')
#ORDERER_LISTEN_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
#ORDERER_HOST=orderer.example.com
EXT_PEER_IP=10.14.128.38

ORDERER_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
ORDERER_PORT=7050
ORDERER_IP=$(kubectl get pods  -l app=orderer-org1-scray-org -o jsonpath='{.items[*].status.podIP}')

kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $ORDERER_IP $ORDERER_HOST $ORDERER_PORT $CHANNEL_NAME 

In [ ]:
#peers.keys()
#[elem for elem in peers.values()]
list(peers.values())[0].__dict__

## invoice usecase

In [123]:
#kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/peer/add-invoice.sh  $CHANNEL_NAME $INVOICE_ID

def get_certificate_string(peername):
    return 'x509::CN=User1@' + peername + '.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.' + peername + '.kubernetes.research.dev.seeburger.de,O=' + peername + '.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'
    #return 'x509::CN=User1@' + peername + '.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.' + peername + '.kubernetes.research.dev.seeburger.de,O=' + peername + '.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE,C=DEdx509::CN=User1@' + peername + '.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.' + peername + '.kubernetes.research.dev.seeburger.de,O=' + peername + '.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'

def chaincode_add_invoice(peer,channel_name,invoice_id,buyer):
    strlist = ['/mnt/conf/peer/add-invoice.sh', channel_name,invoice_id,get_certificate_string(buyer)]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

pernames = list(peers.keys())
chaincode_add_invoice(peers[pernames[0]],channel_name,'046',pernames[1])

/mnt/conf/peer/add-invoice.sh super43 046 x509::CN=User1@lalya-1.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-1.kubernetes.research.dev.seeburger.de,O=lalya-1.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE
/home/jovyan/work/usr/bin/kubectl exec -t lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/add-invoice.sh super43 046 x509::CN=User1@lalya-1.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-1.kubernetes.research.dev.seeburger.de,O=lalya-1.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE


2021-10-07 10:24:09.284 UTC [chaincodeCmd] chaincodeInvokeOrQuery -> INFO 001 Chaincode invoke successful. result: status:200 


''

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -t galya-0-6cc848b49-pv2dn -c scray-peer-cli -- /bin/sh /mnt/conf/peer/add-invoice.sh super13 006 x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US

In [ ]:
get_certificate_string(pernames[1])

#'x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US'

#, 'x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US'

In [122]:
#kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  $CHANNEL_NAME $INVOICE_ID
def chaincode_read_invoice(peer,channel_name):
    strlist = ['/mnt/conf/peer/get-my-invoices.sh', channel_name]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 
chaincode_read_invoice(peers[pernames[0]],channel_name)

/home/jovyan/work/usr/bin/kubectl exec -t lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/get-my-invoices.sh super43


'[{"Key":"invoice1","Record":{"ID":"invoice1","InvoiceOwner":"x509::CN=User1@lalya-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-0.kubernetes.research.dev.seeburger.de,O=lalya-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE","ProductByer":"x509::CN=User1@lalya-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-0.kubernetes.research.dev.seeburger.de,O=lalya-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE"}}]\n'

In [112]:
def chaincode_transfer_invoice(peer,channel_name,invoice_id,new_owner):
    strlist = ['/mnt/conf/peer/transfer_invoice.sh', channel_name,invoice_id,get_certificate_string(new_owner)]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

pernames = list(peers.keys())
chaincode_transfer_invoice(peers[pernames[0]],channel_name,'045',pernames[2])

/mnt/conf/peer/transfer_invoice.sh super43 045 x509::CN=User1@lalya-2.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-2.kubernetes.research.dev.seeburger.de,O=lalya-2.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE
/home/jovyan/work/usr/bin/kubectl exec -t lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/transfer_invoice.sh super43 045 x509::CN=User1@lalya-2.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-2.kubernetes.research.dev.seeburger.de,O=lalya-2.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE


/bin/sh: /mnt/conf/peer/transfer_invoice.sh: Permission denied
command terminated with exit code 126


"Command '['/home/jovyan/work/usr/bin/kubectl', 'exec', '-t', 'lalya-0-7c887cffb8-fpw7h', '-c', 'scray-peer-cli', '--', '/bin/sh', '-c', '/mnt/conf/peer/transfer_invoice.sh super43 045 x509::CN=User1@lalya-2.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-2.kubernetes.research.dev.seeburger.de,O=lalya-2.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE']' returned non-zero exit status 126."

In [114]:
get_certificate_string(pernames[2])
NEW_OWNER='x509::CN=User1@lalya-2.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-2.kubernetes.research.dev.seeburger.de,O=lalya-2.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'
kubectl exec --stdin --tty lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh /mnt/conf/peer/transfer_invoice.sh  super43 045 $NEW_OWNER

'x509::CN=User1@lalya-2.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-2.kubernetes.research.dev.seeburger.de,O=lalya-2.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'

In [119]:
kubectl exec --stdin --tty 'lalya-0-7c887cffb8-fpw7h' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045
kubectl exec --stdin --tty 'lalya-1-7c76dff8c-pzx78' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045
kubectl exec --stdin --tty 'lalya-2-5f8b45dbc4-52tt7' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045


'lalya-0-7c887cffb8-fpw7h'

In [121]:
peers[pernames[2]].pod_name

'lalya-2-5f8b45dbc4-52tt7'

## add channel

In [82]:
orderer.pod_name

'orderer-org1-scray-org-9dbd9fc68-5lrgq'

In [97]:
!/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-9dbd9fc68-5lrgq -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh super43 orderer.example.com 30081

2021-10-07 07:52:53.181 UTC [common.tools.configtxgen] main -> INFO 001 Loading configuration
2021-10-07 07:52:53.204 UTC [common.tools.configtxgen.localconfig] Load -> INFO 002 Loaded configuration: /mnt/conf/orderer/configtx.yaml
2021-10-07 07:52:53.205 UTC [common.tools.configtxgen] doOutputChannelCreateTx -> INFO 003 Generating new channel configtx
2021-10-07 07:52:53.207 UTC [common.tools.configtxgen] doOutputChannelCreateTx -> INFO 004 Writing new channel tx
2021-10-07 07:52:53.274 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-07 07:52:53.341 UTC [cli.common] readBlock -> INFO 002 Expect block, but got status: &{NOT_FOUND}
2021-10-07 07:52:53.345 UTC [channelCmd] InitCmdFactory -> INFO 003 Endorser and orderer connections initialized
2021-10-07 07:52:53.547 UTC [cli.common] readBlock -> INFO 004 Expect block, but got status: &{SERVICE_UNAVAILABLE}
2021-10-07 07:52:53.553 UTC [channelCmd] InitCmdFactory -> INFO 005 Endorser and or

In [ ]:
add_channel('super20')

In [ ]:
name = 'super30'
cmd = toCmd(['/mnt/conf/orderer/scripts/create_channel.sh', name, 'orderer.example.com', '7050'])
cmd, orderer.pod_name

In [ ]:
executePeerCmd(orderer.pod_name, cmd, cli='scray-orderer-cli')

In [ ]:
executePeerCmd(orderer.pod_name, 'peer channel list', cli='scray-orderer-cli')

In [ ]:
cmd1 = ['/mnt/conf/orderer/scripts/create_channel.sh super20 orderer.example.com 7050']
executePeerCmd(orderer.pod_name, cmd1, cli='scray-orderer-cli')

In [ ]:
#cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', 'orderer-org1-scray-org-5f97c57d44-l6d5l', '-c', 'scray-orderer-cli','--', '/bin/sh', '-c','/mnt/conf/orderer/scripts/create_channel.sh super7 orderer.example.com 7050']

#cmd2 = toCmd(['ls','-a'])
cmd2 = toCmd(['/mnt/conf/orderer/scripts/create_channel.sh', 'super7', 'orderer.example.com', '7050'])

cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', 'orderer-org1-scray-org-5f97c57d44-l6d5l', '-c', 'scray-orderer-cli','--', '/bin/sh', '-c',cmd2]


print(toCmd(cmd1))
#subprocess.check_output(cmd1)
subprocess.run(cmd1, stdout = subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -it orderer-org1-scray-org-5f97c57d44-l6d5l -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh super6 orderer.example.com 7050


In [ ]:
orderer.pod_name

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty 'orderer-org1-scray-org-5f97c57d44-j5mbs' -c scray-orderer-cli  -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh 'super' orderer.example.com '7050'

In [ ]:
# Create Peer

peername='dalya8'
addPeer(peername)
peers[peername].__dict__

In [ ]:
executeKubectlCmd(['get', 'pods','-l', 'app=orderer-org1-scray-org','-o','jsonpath=\'{.items[*].status.podIP}\''])  

In [ ]:
peers[peername].__dict__

In [ ]:
orderer.__dict__

## Export channel configuration

In [ ]:
#https://github.com/scray/scray/blob/feature/k8s-peer/projects/invoice-hyperledger-fabric/containers/README.md#export-data

SHARED_FS_HOST='10.14.128.38:30080'
_result = SHARED_FS_HOST + '/channel/configuration/$CHANNEL_NAME/config.json'

#kubectl exec --stdin --tty $ORDERER_POD -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/publish_channel_conf.sh $CHANNEL_NAME $SHARED_FS_HOST

def export_channel_configuration(peer,channel_name,SHARED_FS_HOST):
    strlist = ['/mnt/conf/orderer/scripts/publish_channel_conf.sh', channel_name,SHARED_FS_HOST]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

#_peer = peers[list(peers.keys())[0]]
_peer = orderer
channel_name = 'super'
export_channel_configuration(_peer,channel_name,SHARED_FS_HOST)

In [ ]:
#list(peers)[0]
